In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
data = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
data.describe()

In [ ]:
data['quality'].value_counts().sort_index()/data['quality'].count()*100


Only best of the best! Will choise only 7 and 8 point as "Good wine"

In [ ]:
sns.pairplot(data)

In [ ]:
plt.figure(figsize = (20, 20))
sns.heatmap(data.corr(), cmap="RdYlGn")
plt.show()

In [ ]:
X = data.drop('quality',1)
y = data['quality']

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
sns.countplot(y)

In [ ]:
y = y.apply(lambda x: 1 if x>6 else 0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier



params = {
    'learning_rate': np.arange(0.01,0.1,0.01),
 
}
# Grid Search
gsearch = GridSearchCV(estimator = xgb.XGBClassifier(n_estimators = 1000, learning_rate=0.1, n_jobs=-1), 
 param_grid = params, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
#Fit
gsearch.fit(X_train,y_train)


In [ ]:
 gsearch.best_params_, gsearch.best_score_

In [ ]:
model = xgb.XGBClassifier(n_estimators = 1000, learning_rate=0.03, n_jobs=-1)

model.fit(X_train,y_train,
         early_stopping_rounds=50,
         eval_set=[(X_valid, y_valid)],
         )
pred_prob = model.predict(X_test)

In [ ]:
print(classification_report(y_test, pred_prob))
print(confusion_matrix(y_test, pred_prob))